# SVM Classifier

In [1]:
# general imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# loading in original data sets
df_training = pd.read_csv('../data/archive/train.csv')
df_testing = pd.read_csv('../data/archive/test.csv')

# loading in pca data sets
df_training_pca = pd.read_csv('../data/archive/train_pca.csv')
df_testing_pca = pd.read_csv('../data/archive/test_pca.csv')

In [3]:
# get the normal training and test data

x_train = df_training.iloc[:, :-2]
y_train = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-2]
y_test = df_testing.iloc[:,-1]

# get the pca training and test data

x_train_pca = df_training_pca.iloc[:, :-1]
y_train_pca = df_training_pca.iloc[:,-1]

x_test_pca = df_testing_pca.iloc[:, :-1]
y_test_pca = df_testing_pca.iloc[:,-1]

In [4]:
# sklearn imports

from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.metrics import classification_report

#sorted(sklearn.metrics.SCORERS.keys())

In [20]:
# hyper parameters to testing: kernel, C, gamma

# parameters to test
param_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
best_param_pca = {}

# scoring on accuracy, precicsion_micro and f1_micro
scores = ['accuracy', 'precision_micro', 'f1_micro']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    clf = GridSearchCV(
        svm.SVC(), param_grid, scoring= score
    )
    clf.fit(x_train_pca, y_train_pca)

    print("Best parameters set found on training set:")
    print()
    print(clf.best_params_)
    best_param_pca = clf.best_params_
    print()
    print("Grid scores on training set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
    print()

    print("Detailed classification report using best parameters:")
    print()
    print("The model is trained on the training set.")
    print("The scores are computed on the test set.")
    print()
    y_true, y_pred = y_test_pca, clf.predict(x_test_pca)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for accuracy

Best parameters set found on training set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on training set:

0.885 (+/-0.046) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.786 (+/-0.028) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.898 (+/-0.064) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.885 (+/-0.047) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.906 (+/-0.054) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.897 (+/-0.070) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.908 (+/-0.049) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.902 (+/-0.060) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.900 (+/-0.050) for {'C': 1, 'kernel': 'linear'}
0.897 (+/-0.052) for {'C': 10, 'kernel': 'linear'}
0.897 (+/-0.056) for {'C': 100, 'kernel': 'linear'}
0.896 (+/-0.055) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report using best parameters:

The model is trained on the training set.
The scores are c

In [21]:
%%time

# using the PCA data w/ best hyperparamters
clf = svm.SVC(C= best_param_pca['C'], gamma= best_param_pca['gamma'], kernel= best_param_pca['kernel'])
clf.fit(x_train_pca, y_train_pca)

y_true, y_pred = y_test_pca, clf.predict(x_test_pca)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       537
           2       0.90      0.93      0.91       532
           3       0.92      0.88      0.90       491
           4       0.91      0.92      0.92       496
           5       0.85      0.87      0.86       471
           6       0.91      0.88      0.90       420

    accuracy                           0.92      2947
   macro avg       0.92      0.91      0.92      2947
weighted avg       0.92      0.92      0.92      2947

Wall time: 416 ms


In [22]:
# finding best parameters for the untouched data

best_param_reg = {}

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    clf = GridSearchCV(
        svm.SVC(), param_grid, scoring= score
    )
    clf.fit(x_train, y_train)

    print("Best parameters set found on training set:")
    print()
    print(clf.best_params_)
    best_param_reg = clf.best_params_
    print()
    print("Grid scores on training set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
    print()

    print("Detailed classification report using best parameters:")
    print()
    print("The model is trained on the training set.")
    print("The scores are computed on the test set.")
    print()
    y_true, y_pred = y_test, clf.predict(x_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for accuracy

Best parameters set found on training set:

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

Grid scores on training set:

0.902 (+/-0.044) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.789 (+/-0.030) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.935 (+/-0.047) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.902 (+/-0.045) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.941 (+/-0.046) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.935 (+/-0.052) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.941 (+/-0.047) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.941 (+/-0.048) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.941 (+/-0.051) for {'C': 1, 'kernel': 'linear'}
0.937 (+/-0.051) for {'C': 10, 'kernel': 'linear'}
0.937 (+/-0.048) for {'C': 100, 'kernel': 'linear'}
0.937 (+/-0.048) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report using best parameters:

The model is trained on the training set.
The scores are 

In [23]:
%%time

# using the full data w/ best hyperparameters
clf = svm.SVC(C= best_param_reg['C'], gamma= best_param_reg['gamma'], kernel= best_param_reg['kernel'])
clf.fit(x_train, y_train)

y_true, y_pred = y_test, clf.predict(x_test)
print(classification_report(y_true, y_pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.97      0.89      0.93       491
          STANDING       0.91      0.97      0.94       532
           WALKING       0.94      0.99      0.97       496
WALKING_DOWNSTAIRS       0.99      0.97      0.98       420
  WALKING_UPSTAIRS       0.97      0.95      0.96       471

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947

Wall time: 3.71 s
